# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [131]:
# Configuration and install libraries:
#    - Install plotly with 'conda' and install cufflinks with 'pip'
#    - Upgrade nbformat: pip install --upgrade nbformat

# Import the libraries:
import sys
import re
sys.path.insert(0, "../modules")

import numpy as np
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots
import cufflinks as cf
cf.go_offline()

import module as mod     # functions are include in module.py

In [2]:
# Read dataset.
deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')
deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [3]:
# Add some columns...you'll need them later.
deaths['cause_code'] = deaths['Causa de muerte'].apply(mod.cause_code)
deaths['cause_group'] = deaths['Causa de muerte'].apply(mod.cause_types)
deaths['cause_name'] = deaths['Causa de muerte'].apply(mod.cause_name)

In [4]:
# Lets check the categorical variables.
var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']
categories = mod.cat_var(deaths, var_list)
categories

,categorical_variable,number_of_possible_values,values
0,cause_code,117,"[001-102, 001-008, 001, 002, 003, 004, 005, 00..."
1,cause_name,117,"[I-XXII.Todas las causas, I.Enfermedades infec..."
2,Periodo,39,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."
3,Edad,22,"[Todas las edades, Menos de 1 año, De 1 a 4 añ..."
4,Sexo,3,"[Total, Hombres, Mujeres]"
5,cause_group,2,"[Multiple causes, Single cause]"


In [5]:
# We need also to create a causes table for the analysis.
causes_table = deaths[['cause_code', 'cause_name']].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)
causes_table.head()

,cause_code,cause_name
0,001,Enfermedades infecciosas intestinales
1,001-008,I.Enfermedades infecciosas y parasitarias
2,001-102,I-XXII.Todas las causas
3,002,Tuberculosis y sus efectos tardíos
4,003,Enfermedad meningocócica


__Note:__ I would like to do the exercise with deaths from heart disease. To see this, it's necessary to tilter rows containing a range in 'cause_code'. This way I will be able to see the range of rows I am interested in extracting.

In [6]:
df_filtered = causes_table[causes_table['cause_code'].str.contains(r'\d+-\d+')]   # Regex code: '\d+'=any caracter
df_filtered

,cause_code,cause_name
1,001-008,I.Enfermedades infecciosas y parasitarias
2,001-102,I-XXII.Todas las causas
11,009-041,II.Tumores
45,042-043,III.Enfermedades de la sangre y de los órganos...
48,044-045,"IV.Enfermedades endocrinas, nutricionales y me..."
51,046-049,V.Trastornos mentales y del comportamiento
56,050-052,VI-VIII.Enfermedades del sistema nervioso y de...
60,053-061,IX.Enfermedades del sistema circulatorio
70,062-067,X.Enfermedades del sistema respiratorio
77,068-072,XI.Enfermedades del sistema digestivo


In [29]:
# After that, I can extract the dataset corresponding to cardiac diseases.
df_subset = causes_table.iloc[60:69]
df_subset.head(50)

,cause_code,cause_name
60,053-061,IX.Enfermedades del sistema circulatorio
61,054,Enfermedades hipertensivas
62,055,Infarto agudo de miocardio
63,056,Otras enfermedades isquémicas del corazón
64,057,Insuficiencia cardíaca
65,058,Otras enfermedades del corazón
66,059,Enfermedades cerebrovasculares
67,060,Aterosclerosis
68,061,Otras enfermedades de los vasos sanguíneos


In [ ]:
# And some space for free-style Pandas!!! (e.g.: df['column_name'].unique())
test_df = deaths['cause_code'].unique()
len(test_df)

## Lets make some transformations and show me some insights with Plotly!

Eventhough the dataset is pretty clean, the information is completely denormalized as you could see. F2or that matter a collection of methods (functions) are available in order to generate the tables you might need:

- `row_filter(df, cat_var, cat_values)` => Filter rows by any value or group of values in a categorical variable.

- `nrow_filter(df, cat_var, cat_values)` => The same but backwards. 

- `groupby_sum(df, group_vars, agg_var='Total', sort_var='Total')` => Add deaths by a certain variable.

- `pivot_table(df, col, x_axis, value='Total')`=> Make some pivot tables, you might need them...

__NOTE:__ be aware that the filtering methods can perform a filter at a time. Feel free to perform the filter you need in any way you want or feel confortable with.

### Import and prepare other demographic data: population data by year.

In order to compare each year's data more accurately I will import a dataframe with demographic information. Thus, the values represented in the graph will be percentages of deaths with respect to the population of each year respectively.

In [88]:
# Read dataset.
population = pd.read_csv('../data/datos_poblacion_1971.csv', sep=';', thousands='.')
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65727 entries, 0 to 65726
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Edad simple  65727 non-null  object
 1   Sexo         65727 non-null  object
 2   Periodo      65727 non-null  object
 3   Total        30888 non-null  object
dtypes: object(4)
memory usage: 2.0+ MB


In [40]:
population.head()

,Edad simple,Sexo,Periodo,Total,Periodo_2
0,Todas las edades,Total,1 de enero de 2021,47.400.798,NaT
1,Todas las edades,Total,1 de octubre de 2020,NaN,NaT
2,Todas las edades,Total,1 de julio de 2020,47.344.852,NaT
3,Todas las edades,Total,1 de abril de 2020,NaN,NaT
4,Todas las edades,Total,1 de enero de 2020,47.318.050,NaT


Observing the dataframe it's possible see that there are diferent record to the same year. But some records haven't information. So, I will filter the dataframe by 'Todas las edades' and with the date of January 1 of each year.

The date in the dataset is a string in Spanish, so I create a function to split, clean and translate each element of this column. This function is includes in the 'module.py' file.

In [57]:
def date_transform(date_str):
    months_translation = {
        'enero': 'January',
        'febrero': 'February',
        'marzo': 'March',
        'abril': 'April',
        'mayo': 'May',
        'junio': 'June',
        'julio': 'July',
        'agosto': 'August',
        'septiembre': 'September',
        'octubre': 'October',
        'noviembre': 'November',
        'diciembre': 'December'
    }

    result = date_str.split(' de ')   # Split the string using ' de '
    result[1] = months_translation[result[1]]   # Translate the month from Spanish to English
    
    return ' '.join(result)

In [89]:
# Transform the 'Periodo' column in a date type and create a new column with that information. With the parameters
# errors='coerce', the function to_datetime save NaT (not a time) if can't convert the string to a date. I will check if 
# exist NaT caracter after the transformation.
population['Periodo'] = population['Periodo'].apply(mod.date_transform)
population['Periodo'] = pd.to_datetime(population['Periodo'], format='%d %B %Y', errors='coerce')
population.head()

,Edad simple,Sexo,Periodo,Total
0,Todas las edades,Total,2021-01-01,47.400.798
1,Todas las edades,Total,2020-10-01,NaN
2,Todas las edades,Total,2020-07-01,47.344.852
3,Todas las edades,Total,2020-04-01,NaN
4,Todas las edades,Total,2020-01-01,47.318.050


After that, prepare the dataframe with the data necessary to operate with the death dataset. 

In [96]:
# Filter the dataset and extract in a new dataset the data for each age range, eliminating the total values of the age 
# column.
population_dataset = mod.row_filter(population, 'Edad simple', ['Todas las edades'])
# Extract only the records corresponding to January of each year, sort them by date and reset the index (deleting the 
# previous index).
population_dataset_jan = population_dataset[population_dataset['Periodo'].dt.month == 1]
# Filter this dataset, extracting only the records from 1980 to 2018 (because the death dataset includes data only for 
# this period)
population_dataset_period = population_dataset_jan[population_dataset_jan['Periodo'].dt.year.between(1980, 2018)]\
                         .sort_values(by='Periodo').reset_index(drop=True)

population_dataset_period.head()

,Edad simple,Sexo,Periodo,Total
0,Todas las edades,Hombres,1980-01-01,18.323.513
1,Todas las edades,Mujeres,1980-01-01,19.023.427
2,Todas las edades,Total,1980-01-01,37.346.940
3,Todas las edades,Hombres,1981-01-01,18.467.449
4,Todas las edades,Total,1981-01-01,37.635.389


Now, I'll check if the dataset is correct. First, the dataset doesn't include neither NaT value. On the other hand, the number of records would be 117 (3 x 39 years). 

In [91]:
# Check if there are some NaT in the 'Periodo2' column
numbers_nat = population_dataset_period['Periodo'].isna().any()
print('There is some "NaT" in "Periodo2" column: ', numbers_nat, '\n')

# Info related to dataset
population_dataset_period.info()

There is some "NaT" in "Periodo2" column:  False 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Edad simple  117 non-null    object        
 1   Sexo         117 non-null    object        
 2   Periodo      117 non-null    datetime64[ns]
 3   Total        117 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.8+ KB


Obtain the pivot table to separate the data in the 'Sex' column into three columns: 'Total', 'Hombres'and 'Mujeres'. The result would be a dataset with 39 records.

In [114]:
# Create pivot table to prepare the data for later plotting
population_result_dataset = mod.pivot_table(population_dataset_period, 'Sexo', 'Periodo')
population_result_dataset.info()
population_result_dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Periodo  39 non-null     datetime64[ns]
 1   Hombres  39 non-null     object        
 2   Mujeres  39 non-null     object        
 3   Total    39 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.3+ KB


Sexo,Periodo,Hombres,Mujeres,Total
0,1980-01-01,18.323.513,19.023.427,37.346.940
1,1981-01-01,18.467.449,19.167.940,37.635.389
2,1982-01-01,18.591.423,19.290.449,37.881.873
3,1983-01-01,18.695.393,19.394.759,38.090.151
4,1984-01-01,18.775.089,19.477.809,38.252.899


In [115]:
# I will have date records of the same day and month, but of different year. With this method only the year will be saved. 
# This will be useful to be able to merge later with other dataframes.
population_result_dataset['Periodo'] = population_result_dataset['Periodo'].dt.year
# Transform the 'Hombres', 'Mujeres' and 'Total' columns from string to int. This is necessary in order to be able to 
# perform operations on the data afterwards. Before making the transformation, the dots in the numbers, which are thousands 
# separators, must be removed.
columns_transform = ['Hombres', 'Mujeres', 'Total']
population_result_dataset[columns_transform] = population_result_dataset[columns_transform]\
                                            .apply(lambda x: x.str.replace('.', '').astype(int))
population_result_dataset.head()

Sexo,Periodo,Hombres,Mujeres,Total
0,1980,18323513,19023427,37346940
1,1981,18467449,19167940,37635389
2,1982,18591423,19290449,37881873
3,1983,18695393,19394759,38090151
4,1984,18775089,19477809,38252899


### First Analysis

To start exploring the data and see how it evolves over time:

In [85]:
# Filter the dataset and extract only the data corresponding to all sexes and ages
dataset = mod.row_filter(deaths, 'cause_name', ['IX.Enfermedades del sistema circulatorio'])
dataset_edad_total = mod.row_filter(dataset, 'Edad', ['Todas las edades'])
#dataset_edad_sexo_total = mod.row_filter(dataset_edad_total, 'Sexo', ['Total'])

# Create pivot table to prepare the data for later plotting
dataset_edad_total_pivot = mod.pivot_table(dataset_edad_total, 'Sexo', 'Periodo')
dataset_edad_total_pivot.info()
dataset_edad_total_pivot.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Periodo  39 non-null     int64
 1   Hombres  39 non-null     int64
 2   Mujeres  39 non-null     int64
 3   Total    39 non-null     int64
dtypes: int64(4)
memory usage: 1.3 KB


Sexo,Periodo,Hombres,Mujeres,Total
0,1980,59121,66766,125887
1,1981,59668,67918,127586
2,1982,57932,66054,123986
3,1983,60172,70359,130531
4,1984,58834,67970,126804


Calculate the percentage of people who have died due to heart disease in relation to the country's population on January 1 of each year. Before doing this operation, it's necessary to merge the two dataframe.

In [122]:
# Merge the two dataframes and inclucludes the suffixes in the function to diferenciate between variable names.
popul_vs_death = pd.merge(population_result_dataset, dataset_edad_total_pivot, on='Periodo'
                          , suffixes=('_poblacion', '_muertes'))

# Calculate the percentage
popul_vs_death['Hombres_%'] = (popul_vs_death['Hombres_muertes'] / popul_vs_death['Hombres_poblacion']) * 100
popul_vs_death['Mujeres_%'] = (popul_vs_death['Mujeres_muertes'] / popul_vs_death['Mujeres_poblacion']) * 100
popul_vs_death['Total_%'] = (popul_vs_death['Total_muertes'] / popul_vs_death['Total_poblacion']) * 100
popul_vs_death.head()

Sexo,Periodo,Hombres_poblacion,Mujeres_poblacion,Total_poblacion,Hombres_muertes,Mujeres_muertes,Total_muertes,Hombres_%,Mujeres_%,Total_%
0,1980,18323513,19023427,37346940,59121,66766,125887,0.322651,0.350967,0.337074
1,1981,18467449,19167940,37635389,59668,67918,127586,0.323098,0.354331,0.339005
2,1982,18591423,19290449,37881873,57932,66054,123986,0.311606,0.342418,0.327296
3,1983,18695393,19394759,38090151,60172,70359,130531,0.321855,0.362773,0.342690
4,1984,18775089,19477809,38252899,58834,67970,126804,0.313362,0.348961,0.331489


The pivot table has only 39 rows due to the pivot operation create two columns: one for male deaths, one for female deaths and other for both.

In [137]:
# Shows the graph of number of deaths per year.
popul_vs_death.iplot(kind='bar',
               x='Periodo',
               y='Total_%',
               title='Percentage of deaths due to heart disease in relation to the total population.',
               yTitle='Deaths Vs. population (%)',
               xTitle='Years')

This graph shows that the evolution of deaths between 1980 and 2018 has decreased appreciably. 

It should be noted, that prior to this graph the number of deaths for each year had been plotted, and in that case the graph was stable over the years and no decrease was observed.

Although it is predictable that heart disease affects men more than women, we are going to add a comparative graph to corroborate this.

In [63]:
# Extract the dataset with data for all age ranges, separating between males and females. 
#dataset_edad_total_hym = dataset_edad_total[dataset_edad_total['Sexo'].isin(['Hombres', 'Mujeres'])]

In [140]:
popul_vs_death.iplot(kind='bar',
                   x='Periodo',
                   y=['Hombres_%', 'Mujeres_%'],
                   title='Percentage of deaths due to heart disease in relation to the total population.',
                   yTitle='Deaths Vs. population (%)',
                   xTitle='Years')

Finally, there is a higher number of deaths in the female population than in the male population, although it is not much higher. The trend over the years is quite stable and has decreased appreciably for both.

This surprises me, since heart disease in general has a higher incidence in the male population and therefore I think the mortality figures should be higher in men than in women.

After this representation

### Second Analysis

Next, I want to observe for which age range there is a higher incidence for both males and females. In order to study these data in greater depth.

In [61]:
# Filter the dataset and extract in a new dataset the data for each age range, eliminating the total values of the age 
# column.
dataset2 = mod.row_filter(deaths, 'cause_name', ['IX.Enfermedades del sistema circulatorio'])
dataset_rangos_edad.head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
39,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2018,1027,001-102,Multiple causes,I-XXII.Todas las causas
40,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2017,1092,001-102,Multiple causes,I-XXII.Todas las causas
41,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2016,1120,001-102,Multiple causes,I-XXII.Todas las causas
42,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2015,1139,001-102,Multiple causes,I-XXII.Todas las causas
43,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2014,1232,001-102,Multiple causes,I-XXII.Todas las causas


In [13]:
# Check the age ranges in the dataframe
rangos = dataset_rangos_edad['Edad'].unique()
print('Age ranges info: ', rangos)
print('The number of age ranges that there are: ', len(rangos))

Age ranges info:  ['Menos de 1 año' 'De 1 a 4 años' 'De 15 a 19 años  ' 'De 20 a 24 años'
 'De 25 a 29 años' 'De 30 a 34 años' 'De 35 a 39 años' 'De 40 a 44 años'
 'De 45 a 49 años' 'De 50 a 54 años' 'De 55 a 59 años' 'De 60 a 64 años'
 'De 65 a 69 años' 'De 70 a 74 años  ' 'De 75 a 79 años  '
 'De 80 a 84 años  ' 'De 85 a 89 años  ' 'De 90 a 94 años  '
 '95 y más años' 'De 5 a 9 años' 'De 10 a 14 años  ']
The number of age ranges that there are:  21


In order to represent the total deaths for each age range for males and females, it is necessary to sum the values for all years for each age range. Since there are 21 age ranges, the resulting matrix should have 21 records. With the pivot table function, you can perform the sum and also generate two columns, one for the male and one for the female values.

In [14]:
# Create pivot table to prepare the data for later plotting
#dataset_rangos_edad_sum = dataset_rangos_edad.groupby(['Edad', 'Sexo'])['Total'].sum().reset_index()
dataset_rangos_edad_pivot = mod.pivot_table(dataset_rangos_edad, 'Sexo', 'Edad', 'Total')
dataset_rangos_edad_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Edad     21 non-null     object 
 1   Hombres  21 non-null     float64
 2   Mujeres  21 non-null     float64
 3   Total    19 non-null     float64
dtypes: float64(3), object(1)
memory usage: 800.0+ bytes


In [126]:
dataset_rangos_edad_pivot.head(21)

Sexo,Edad,Hombres,Mujeres
0,95 y más años,423986,1252651
1,De 1 a 4 años,39155,30089
2,De 10 a 14 años,33934,21759
3,De 15 a 19 años,102685,40327
4,De 20 a 24 años,165108,51004
5,De 25 a 29 años,193016,63587
6,De 30 a 34 años,229362,83709
7,De 35 a 39 años,273285,116392
8,De 40 a 44 años,366081,168083
9,De 45 a 49 años,541345,248025


The values in the age column are not sorted correctly, because the names of the first and last records do not follow the same structure as the rest. In order for the names of the age ranges to be ordered correctly in the graph, it is necessary to change the name of the range '95 and over' and the range 'Less than 1 year old'.

On the other hand, I will add a '0' in the names: 'De 1 a 4 años' and 'De 5 a 9 años'. This way the will be sorted correctly and the records will look like: 'De 01 a 04 años' and 'De 05 a 09 años'. 

In [15]:
print('First record: ', dataset_rangos_edad_pivot['Edad'][0])
print('Second record: ', dataset_rangos_edad_pivot['Edad'][1])
print('10 record:', dataset_rangos_edad_pivot['Edad'][10])
print('Last record: ', dataset_rangos_edad_pivot['Edad'][20])

First record:  95 y más años
Second record:  De 1 a 4 años
10 record: De 5 a 9 años
Last record:  Menos de 1 año


In [16]:
dataset_rangos_edad_pivot['Edad'][0] = 'De 95 o más años'
dataset_rangos_edad_pivot['Edad'][1] = 'De 01 a 04 años'
dataset_rangos_edad_pivot['Edad'][10] = 'De 05 a 09 años'
dataset_rangos_edad_pivot['Edad'][20] = 'De 00 a 01 año'

In [18]:
dataset_rangos_edad_pivot_sort = dataset_rangos_edad_pivot.sort_values(by='Edad').reset_index(drop=True)
#dataset_rangos_edad_pivot_sort.head(21)

The graph with the data obtained can be seen below:

In [19]:
dataset_rangos_edad_pivot_sort.iplot(kind='bar',
                   x='Edad',
                   y=['Hombres', 'Mujeres'],
                   title='Deaths due to heart disease',
                   yTitle='No. of deaths',
                   xTitle='Years')

For the female population, the data have an exponential trend up to the 85-89 age range. From then on, it starts to decrease. On the other hand, for the male population, the values are grouped as in steps of three by three. From 50 to 64 it increases with respect to previous ages. There is another large increase from 65 to 84, where it begins to decrease.

For the data corresponding to men, the numbers of deaths begin to grow more rapidly after the 50 years. This is logical, because men have a higher incidence of heart disease at this age.

It is possible to see that the numbers of deaths from heart disease are higher in men than in women in practically all age ranges up to the age of 80, where this trend is reversed and the numbers of deaths in men from heart disease fall notably with respect to the numbers in women. This may be due to the fact that women in Spain have an average life expectancy 5 years longer than men, which may be one reason why mortality figures for this type of disease are much lower in men over 85 years of age. 

This large increase in deaths in women over the age of 80 may justify the fact that in the above graph the total number of deaths in women is higher than in men.

Next, plot the above graph but for the year 2018 only. To do this I have to follow the same steps done above, but applying a filter to keep only the 2018 data.

In [226]:
# Filter the dataset and extract only the data corresponding to all sexes and ages
dataset_2018 = mod.row_filter(deaths, 'cause_name', ['IX.Enfermedades del sistema circulatorio'])
dataset_2018 = mod.row_filter(dataset_2018, 'Periodo', [2018])
dataset_2018 = dataset_2018[dataset_2018['Edad'] != 'Todas las edades']

# Create pivot table to prepare the data for later plotting
dataset_pivot_2018 = mod.pivot_table(dataset_2018, 'Sexo', 'Edad', 'Total')
dataset_pivot_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Edad     21 non-null     object
 1   Hombres  21 non-null     int64 
 2   Mujeres  21 non-null     int64 
 3   Total    21 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 800.0+ bytes


As expected, the dataset has 22 records, as in the previous case, since there are 22 age ranges. As before, the dataset is corrected and sorted.

In [227]:
# Correct and sort the columns
dataset_pivot_2018['Edad'][0] = 'De 95 o más años'
dataset_pivot_2018['Edad'][1] = 'De 01 a 04 años'
dataset_pivot_2018['Edad'][10] = 'De 05 a 09 años'
dataset_pivot_2018['Edad'][20] = 'De 00 a 01 año'

# Sort the columns by 'Edad'
dataset_pivot_2018 = dataset_pivot_2018.sort_values(by='Edad').reset_index(drop=True)
#dataset_pivot_2018.head(22)

In [228]:
dataset_pivot_2018.iplot(kind='bar',
                   x='Edad',
                   y=['Hombres', 'Mujeres'],
                   title='Deaths due to heart disease in 2018',
                   yTitle='No. of deaths',
                   xTitle='Years')

A similar trend can be observed for all years. In this case, the trend of the values for men is exponential up to the age of 80 years.

Probably, since the life expectancy in 2018 is higher than in the total 39 years, in the range from 80 to 84 the number of deaths is practically the same for men as for women. Thereafter, the number of female deaths is higher than the number of male deaths.

On the other hand, deaths in the 95 and older age range are lower, since in general for this age range the number of deaths will be lower for any type of disease.

### Import and prepare other demographic data: life expectancy by year.

With these data, it will be possible to test the hypothesis that up to 79 years of age, the number of deaths among men is higher than among women. From the age of 80 onwards, the opposite is true. 
I have downloaded three datasets from the INE website, one with total data by sex, one with data for men and one for women.

In [224]:
# Read each dataset, extract only the columns I am interested in for analysis and convert the life expectancy column from 
# string to float. Finally, for each dataset, the records between 1980 and 2018 are selected, eliminating the rest.
# Total
esp_vida_total = pd.read_csv('../data/esperanza_vida_total.csv', sep=';', thousands='.')
esp_vida_total = esp_vida_total[['PERIODO', 'VALOR']]\
                .rename(columns={'PERIODO': 'Periodo', 'VALOR': 'Esp_vida_total'})
esp_vida_total['Esp_vida_total'] = esp_vida_total['Esp_vida_total'].str.replace(',', '.').astype(float)
esp_vida_total = esp_vida_total[(esp_vida_total['Periodo'] >= 1980) & (esp_vida_total['Periodo'] <= 2018)]\
                .reset_index(drop=True)

# Men
esp_vida_hombres = pd.read_csv('../data/esperanza_vida_hombres.csv', sep=';', thousands='.')
esp_vida_hombres = esp_vida_hombres[['PERIODO', 'VALOR']]\
                .rename(columns={'PERIODO': 'Periodo', 'VALOR': 'Esp_vida_hombres'})
esp_vida_hombres['Esp_vida_hombres'] = esp_vida_hombres['Esp_vida_hombres'].str.replace(',', '.').astype(float)
esp_vida_hombres = esp_vida_hombres[(esp_vida_hombres['Periodo'] >= 1980) & (esp_vida_hombres['Periodo'] <= 2018)]\
                .reset_index(drop=True)

# Women
esp_vida_mujeres = pd.read_csv('../data/esperanza_vida_mujeres.csv', sep=';', thousands='.')
esp_vida_mujeres = esp_vida_mujeres[['PERIODO', 'VALOR']]\
                .rename(columns={'PERIODO': 'Periodo', 'VALOR': 'Esp_vida_mujeres'})
esp_vida_mujeres['Esp_vida_mujeres'] = esp_vida_mujeres['Esp_vida_mujeres'].str.replace(',', '.').astype(float)
esp_vida_mujeres = esp_vida_mujeres[(esp_vida_mujeres['Periodo'] >= 1980) & (esp_vida_mujeres['Periodo'] <= 2018)]\
                .reset_index(drop=True)


esp_vida_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Periodo         39 non-null     int64  
 1   Esp_vida_total  39 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 752.0 bytes


Since life expectancy is calculated at birth, there will be one value in this dataset for each year included in the dataset. Since only the 1980-2018 working range has been selected, each of the datasets will have 39 records.

A graph with the evolution of life expectancy is represented below:

In [231]:
# To plot a graph with the men and women expectancy data, it's necessary to obtain a new dataset with both data toguether
esp_vida_hym = pd.merge(esp_vida_hombres, esp_vida_mujeres, on='Periodo')
# Next, I will merge the dataset below with esp_vida_total dataset
esp_vida_result = pd.merge(esp_vida_hym, esp_vida_total, on='Periodo')\
                .rename(columns={'Esp_vida_hombres': 'Hombres', 'Esp_vida_mujeres': 'Mujeres', 'Esp_vida_total': 'Total'})
esp_vida_result.head()

,Periodo,Hombres,Mujeres,Total
0,2018,80.398753,85.827440,83.150685
1,2017,80.321801,85.707669,83.054179
2,2016,80.264826,85.818135,83.078504
3,2015,79.889368,85.394006,82.673280
4,2014,80.095819,85.631136,82.906200


In [233]:
esp_vida_result.iplot(kind='line',
                   x='Periodo',
                   y=['Hombres', 'Mujeres', 'Total'],
                   title='Life expectancy in Spain (1980 - 2018)',
                   yTitle='Life expectancy',
                   xTitle='Years')

As mentioned earlier, the life expectancy of women is higher than that of men. In 1980 the difference was 6 years while in 2018 it is 5 years, so although it is still much higher, it is progressively decreasing. 

In the 39 years represented, there has been an increase of 8 years in life expectancy in Spain.

### Third Analysis

To check this, I will represent the previous graph, but only with the age ranges from 0 to 79 years. If this hypothesis is true, deaths in men should be higher than in women. 

In [20]:
# Extract in a new dataset the data for each age range from 0 to 79 years.
dataset3 = mod.row_filter(deaths, 'cause_name', ['IX.Enfermedades del sistema circulatorio'])
dataset_rangos_edad_79 = deaths[(dataset3['Sexo'].isin(['Hombres', 'Mujeres'])) & (deaths['Edad'] != 'Todas las edades')\
                            & (deaths['Edad'] != 'De 80 a 84 años  ') & (deaths['Edad'] != 'De 85 a 89 años  ')\
                            & (deaths['Edad'] != 'De 90 a 94 años  ') & (deaths['Edad'] != '95 y más años')]
# Another way: (~deaths['Edad'].isin(list_with_age_ranges_to_delete))
dataset_rangos_edad_79.head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
39,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2018,1027,001-102,Multiple causes,I-XXII.Todas las causas
40,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2017,1092,001-102,Multiple causes,I-XXII.Todas las causas
41,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2016,1120,001-102,Multiple causes,I-XXII.Todas las causas
42,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2015,1139,001-102,Multiple causes,I-XXII.Todas las causas
43,001-102 I-XXII.Todas las causas,Total,Menos de 1 año,2014,1232,001-102,Multiple causes,I-XXII.Todas las causas


In [23]:
# Check the age ranges in the dataframe
rangos_79 = dataset_rangos_edad_79['Edad'].unique()
print('Age ranges info: ', rangos_79)
print('The number of age ranges that there are: ', len(rangos_79))

Age ranges info:  ['Menos de 1 año' 'De 1 a 4 años' 'De 15 a 19 años  ' 'De 20 a 24 años'
 'De 25 a 29 años' 'De 30 a 34 años' 'De 35 a 39 años' 'De 40 a 44 años'
 'De 45 a 49 años' 'De 50 a 54 años' 'De 55 a 59 años' 'De 60 a 64 años'
 'De 65 a 69 años' 'De 70 a 74 años  ' 'De 75 a 79 años  ' 'De 5 a 9 años'
 'De 10 a 14 años  ']
The number of age ranges that there are:  17


In [144]:
# Create pivot table to prepare the data for later plotting
dataset_edad_total_pivot = mod.pivot_table(dataset_rangos_edad_79, 'Sexo', 'Periodo')
dataset_edad_total_pivot.info()
dataset_edad_total_pivot.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Periodo  39 non-null     int64
 1   Hombres  39 non-null     int64
 2   Mujeres  39 non-null     int64
 3   Total    39 non-null     int64
dtypes: int64(4)
memory usage: 1.3 KB


Sexo,Periodo,Hombres,Mujeres,Total
0,1980,81005,26165,177317
1,1981,75398,73782,140754
2,1982,58607,43068,146647
3,1983,84390,62469,119298
4,1984,46925,46236,95959


Calculate the percentage of people who have died due to heart disease in relation to the country's population on January 1 of each year. Before doing this operation, it's necessary to merge the two dataframe.

In [145]:
# Merge the two dataframes and inclucludes the suffixes in the function to diferenciate between variable names.
popul2_vs_death = pd.merge(population_result_dataset, dataset_edad_total_pivot, on='Periodo'
                          , suffixes=('_poblacion', '_muertes'))

# Calculate the percentage
popul2_vs_death['Hombres_%'] = (popul2_vs_death['Hombres_muertes'] / popul2_vs_death['Hombres_poblacion']) * 100
popul2_vs_death['Mujeres_%'] = (popul2_vs_death['Mujeres_muertes'] / popul2_vs_death['Mujeres_poblacion']) * 100
popul2_vs_death['Total_%'] = (popul2_vs_death['Total_muertes'] / popul2_vs_death['Total_poblacion']) * 100
popul2_vs_death.head()

Sexo,Periodo,Hombres_poblacion,Mujeres_poblacion,Total_poblacion,Hombres_muertes,Mujeres_muertes,Total_muertes,Hombres_%,Mujeres_%,Total_%
0,1980,18323513,19023427,37346940,81005,26165,177317,0.442082,0.137541,0.474783
1,1981,18467449,19167940,37635389,75398,73782,140754,0.408275,0.384924,0.373994
2,1982,18591423,19290449,37881873,58607,43068,146647,0.315237,0.223261,0.387117
3,1983,18695393,19394759,38090151,84390,62469,119298,0.451395,0.322092,0.313199
4,1984,18775089,19477809,38252899,46925,46236,95959,0.249932,0.237378,0.250854


In [148]:
popul2_vs_death.iplot(kind='bar',
                   x='Periodo',
                   y=['Hombres_%', 'Mujeres_%'],
                   title='Percentage of deaths due to heart disease in relation to the total population (0-79 years old)',
                   yTitle='Deaths Vs. population (%)',
                   xTitle='Years')

As can be seen, in most of years, male deaths are much higher than in the case of females. And there is also much more variability, so it can be said that the data on deaths in the age range of 80 years and older has a lot of weight in the final statistics. 

It would be necessary to study in greater depth years such as 1991 or 2001, in which the death rate for men under 80 years of age decreased considerably.